<a href="https://colab.research.google.com/github/eliesg/portfolio-data/blob/main/Mon_notebook_sandbox_(bquxjob_2e7148d0_1887353f537).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'ga-integration-1470646678388' # Project ID inserted based on the query results selected to explore
location = 'EU' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

## Reference SQL syntax from the original job
Use the ```jobs.query```
[method](https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs/query) to
return the SQL syntax from the job. This can be copied from the output cell
below to edit the query now or in the future. Alternatively, you can use
[this link](https://console.cloud.google.com/bigquery?j=ga-integration-1470646678388:EU:bquxjob_7bc03005_188728b60c3)
back to BigQuery to edit the query within the BigQuery user interface.

In [2]:
# Running this code will display the query used to generate your previous job

job = client.get_job('bquxjob_2e7148d0_1887353f537') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT
  event_date,
  event_name,
  platform_airline_code,
  platform_country,
  count(*) as total_events,
  countif(event_name = "page_view") as total_pageviews,
  count(distinct user_pseudo_id) as users,
  count(distinct session_id) as sessions,
FROM `ga-integration-1470646678388.analytics_322130036.unnested_events`
WHERE
  event_date >= "2023-05-27"
  and event_name in ('web_event', 'page_view')
GROUP BY
  1,2,3,4
ORDER BY
  total_events DESC
LIMIT 5000


# Result set loaded from BigQuery job as a DataFrame
Query results are referenced from the Job ID ran from BigQuery and the query
does not need to be re-run to explore results. The ```to_dataframe```
[method](https://googleapis.dev/python/bigquery/latest/generated/google.cloud.bigquery.job.QueryJob.html#google.cloud.bigquery.job.QueryJob.to_dataframe)
downloads the results to a Pandas DataFrame by using the BigQuery Storage API.

To edit query syntax, you can do so from the BigQuery SQL editor or in the
```Optional:``` sections below.

In [3]:
# Running this code will read results from your previous job

results = job.to_dataframe()
results

,event_date,event_name,platform_airline_code,platform_country,total_events,total_pageviews,users,sessions
0,2023-05-29,web_event,af,fr,4538098,0,217975,264125
1,2023-05-28,web_event,af,fr,3469043,0,170734,204181
2,2023-05-27,web_event,af,fr,3306319,0,164285,197680
3,2023-05-29,page_view,af,fr,2479094,2479094,254900,318295
4,2023-05-28,page_view,af,fr,1937865,1937865,202609,250631
...,...,...,...,...,...,...,...,...
1205,2023-05-29,web_event,kl,ps,1,0,1,1
1206,2023-05-28,page_view,kl,dz,1,1,1,1
1207,2023-05-27,web_event,af,ph,1,0,1,1
1208,2023-05-27,page_view,kl,ve,1,1,1,1


## Show descriptive statistics using describe()
Use the ```pandas DataFrame.describe()```
[method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
to generate descriptive statistics. Descriptive statistics include those that
summarize the central tendency, dispersion and shape of a dataset’s
distribution, excluding ```NaN``` values. You may also use other Python methods
to interact with your data.

In [4]:
results.describe()

,total_events,total_pageviews,users,sessions
count,1210.0,1210.0,1210.0,1210.0
mean,47064.389256,16677.671074,3376.27686,4100.803306
std,245102.423551,118800.692459,16330.678433,20048.873638
min,1.0,0.0,1.0,1.0
25%,1189.0,0.0,108.0,128.5
50%,5455.0,1.0,466.5,553.0
75%,20563.0,4375.25,1487.5,1775.25
max,4538098.0,2479094.0,254900.0,318295.0


## Mes petits tests (Datacamp)

In [5]:
pivot = results.pivot_table(values="users", index="platform_country")
pivot_sorted = pivot.sort_values(by="users", ascending=False)
print(pivot_sorted)

                          users
platform_country               
fr                104162.666667
us                      54803.5
nl                 39340.583333
gb                 19626.833333
de                 12575.166667
...                         ...
ba                          1.0
ps                          1.0
ai                          1.0
ag                          1.0
py                          1.0

[139 rows x 1 columns]
